## Project: Question-Answering on Private Documents

In [1]:
import os 
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

True

In [2]:
pip install pypdf -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
def load_document(file):
    from langchain.document_loaders import PyPDFLoader
    print(f'Loading {file}....')
    loader=PyPDFLoader(file)
    data= loader.load()
    return data

In [15]:
def chunck_data(data,chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks=text_splitter.split_documents(data)
    return chunks

In [35]:
data= load_document('files/Manual del Dungeon Master Completo 5E.pdf')
chunks=chunck_data(data,1500)
print(len(chunks))

Loading files/Manual del Dungeon Master Completo 5E.pdf....
1079


## Embedding y subirlo a Pinecone


In [41]:
def insert_or_fetch_embeddings(index_name):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings import OpenAIEmbeddings
    embeddings= OpenAIEmbeddings()
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    if index_name  in pinecone.list_indexes():
         print(f'El indice{index_name} ya existe, cargando embeddings...', end='')
         vector_store= Pinecone.from_existing_index(index_name, embeddings)
         print('OK')
    else:
          print(f'Creando el indice {index_name}...', end='')
          pinecone.create_index(index_name, dimension=1536, metric='cosine')
          vector_store=Pinecone.from_documents(chunks,embeddings,index_name=index_name)
          print('OK')

    return vector_store        
          



In [20]:
#Borrar indices
def delete_pinecone_index(index_name='all'):
    import pinecone
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    if index_name == 'all':
        indexes= pinecone.list_indexes()
        print('Se van a borrar todos los indices...')
        for index in indexes:
            pinecone.delete_index(index)
            print('Done')
        else:
            print(f'Borrando indice  {index_name}...', end='')
            pinecone.delete_index(index_name)
            print('Ok')

In [ ]:
delete_pinecone_index()

In [42]:
index_name='document-dnd'
vector_store=insert_or_fetch_embeddings(index_name)

El indicedocument-dnd ya existe, cargando embeddings...OK


### Se vienen preguntas 

In [44]:
def ask_and_get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm= ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriver=vector_store.as_retriever(search_type='similarity',search_kwargs={'k': 3})

    chain=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriver)
    answer=chain.run(q)
    return answer
def ask_with_memory(vector_store,q,chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI
    llm= ChatOpenAI( temperature=1)
    retriver=vector_store.as_retriever(search_type='similarity',search_kwargs={'k': 3})
    crc=ConversationalRetrievalChain.from_llm(llm,retriver)
    result=crc({'question':q,'chat_history':chat_history})
    chat_history.append((q,result['answer']))

    return result, chat_history

In [30]:
q='¿Sobre que trata el documento?'
answer= ask_and_get_answer(vector_store,q)
print(answer)

El documento trata sobre las bases de la vida de aventurero en el juego de Dungeons & Dragons. Cubre desde las mecánicas de movimiento hasta las complejidades de la interacción social. También incluye las reglas de descanso y una discusión sobre las actividades que los personajes pueden realizar entre aventuras. El juego sigue un ritmo natural, donde el DM describe el entorno, los jugadores describen lo que quieren hacer y el DM narra los resultados de sus acciones. Además, el documento incluye ideales, vínculos y defectos que pueden ayudar a los jugadores a desarrollar sus personajes.


In [43]:
import time
i= 1
print('Escribe Quit o Exit para parar la aplicacion')
while True:
    q= input(f'Pregunta #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Chao Pescao')
        time.sleep(2)
        break
    answer = ask_and_get_answer(vector_store,q) 
    print(f'\nRespuesta:{answer}')
    print(f'\n{"-" * 50}\n')

Escribe Quit o Exit para parar la aplicacion

Respuesta:El Dungeon Master (DM) es el narrador del juego y el árbitro. Su función principal es crear y dirigir las aventuras para los personajes. Esto incluye describir el entorno, los personajes no jugadores y los eventos que ocurren. Además, el DM resuelve las acciones de los jugadores y narra los resultados de sus acciones. También tiene la capacidad de improvisar y reaccionar a las decisiones de los jugadores para mantener el juego emocionante e inesperado.

--------------------------------------------------


Respuesta:En los mundos de D&D, puedes encontrar una amplia variedad de razas. Algunas de las razas más comunes incluyen:

1. Humanos: Los humanos son una raza adaptable y diversa. Son conocidos por su variedad de apariencias y culturas.

2. Enanos: Los enanos son una raza resistente y experta en la artesanía. Son conocidos por su habilidad en la minería y la forja.

3. Elfos: Los elfos son una raza elegante y longeva. Son maestr

### Running Code

In [14]:
def print_embedding_cost(text):
    import tiktoken
    enc= tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens= sum([len(enc.encode(page.page_content))for page in text])
    print(f'Numero total de Tokens{total_tokens}')
    print(f'Mostrando coste en USD: {total_tokens / 1000 * 0.0004:.6f}')

print_embedding_cost(chunks)

Numero total de Tokens172808
Mostrando coste en USD: 0.069123
